In [26]:
import os
import json
import pandas as pd

folder_path = '/home/jupyter-23521027/refresh-bert/data/liputan6_data/canonical/test' 

file_list = os.listdir(folder_path)
json_data_list = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name) 

    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            json_data_list.append(json_data)

df = pd.DataFrame(json_data_list)

In [27]:
df

,id,url,clean_article,clean_summary,extractive_summary
0,19962,https://www.liputan6.com/news/read/19962/gas-b...,"[[Liputan6, ., com, ,, Jakarta, :, Untuk, meme...","[[Pertamina, akan, menyalurkan, gas, alam, dar...","[0, 4]"
1,23319,https://www.liputan6.com/news/read/23319/pelaj...,"[[Liputan6, ., com, ,, Yogyakarta, :, Sekitar,...","[[Ratusan, siswa, Sekolah, Menangah, Umum, 6, ...","[1, 2]"
2,18513,https://www.liputan6.com/news/read/18513/bamba...,"[[Liputan6, ., com, ,, Jakarta, :, Ekspektasi,...","[[Tim, Ekonomi, Pemerintah, yang, sedang, beru...","[0, 6]"
3,23951,https://www.liputan6.com/news/read/23951/bengk...,"[[Liputan6, ., com, ,, Jakarta, :, Perampokan,...","[[Bengkel, Toyota, Astra, dan, Indomaret, di, ...","[0, 1]"
4,13167,https://www.liputan6.com/news/read/13167/bentr...,"[[Liputan6, ., com, ,, Jakarta, :, Bentrokan, ...","[[Bentrokan, antara, mahasiswa, satu, kampus, ...","[0, 7]"
...,...,...,...,...,...
10967,15906,https://www.liputan6.com/news/read/15906/soepa...,"[[Liputan6, ., com, ,, Jakarta, :, Ketua, DPR,...","[[Kepergian, Baharuddin, Lopa, membuat, Akbar,...","[0, 3]"
10968,19241,https://www.liputan6.com/news/read/19241/golka...,"[[Liputan6, ., com, ,, Jakarta, :, Partai, Gol...","[[Ketua, Umum, DPP, Partai, Golkar, Akbar, Tan...","[2, 3]"
10969,20408,https://www.liputan6.com/news/read/20408/pemer...,"[[Liputan6, ., com, ,, Jakarta, :, Pemerintah,...","[[Pemerintah, akan, memberlakukan, sistem, UMR...","[1, 4, 5]"
10970,26145,https://www.liputan6.com/news/read/26145/pemer...,"[[Liputan6, ., com, ,, Medan, :, Pemerintah, t...","[[Biro, perjalanan, haji, yang, menggunakan, p...","[1, 5]"


In [28]:
df['sentences'] = df['clean_article'].apply(lambda x: [' '.join(sentence) for sentence in x])

In [29]:
df.head(1)

,id,url,clean_article,clean_summary,extractive_summary,sentences
0,19962,https://www.liputan6.com/news/read/19962/gas-b...,"[[Liputan6, ., com, ,, Jakarta, :, Untuk, meme...","[[Pertamina, akan, menyalurkan, gas, alam, dar...","[0, 4]","[Liputan6 . com , Jakarta : Untuk memenuhi ken..."


In [23]:
import nltk
from nltk.tokenize import word_tokenize
from datetime import datetime

# Preprocessing step: tokenization and cleaning

# Define the context window size
window_size = 2

# Open the text file
with open('/home/jupyter-23521027/refresh-bert/data/indo4b/dataset_all_uncased_blankline.txt', 'r') as file:
    # Iterate over each line in the file
    now = datetime.now()
    word_pairs = []

    for l, line in enumerate(file):
        # Tokenize the line into words
        words = word_tokenize(line)
        
        # Generate word pairs
        for i, target_word in enumerate(words):
            for j in range(i - window_size, i + window_size + 1):
                if j != i and j >= 0 and j < len(words):
                    context_word = words[j]
                    word_pairs.append((target_word, context_word))
                    
        if ( l + 1 ) % 100000 == 0:
            print(f"processed {l+1} files in : {datetime.now() - now}")
            now = datetime.now()
        
        if l == 500000:
            break

# Assuming you have word_pairs as a list of word pairs

# Create word-to-index and index-to-word mappings
words = set([pair[0] for pair in word_pairs] + [pair[1] for pair in word_pairs])
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

# Convert word pairs to numerical representations
numeric_pairs = [(word_to_index[pair[0]], word_to_index[pair[1]]) for pair in word_pairs]

processed 100000 files in : 0:00:13.681185
processed 200000 files in : 0:00:11.630862
processed 300000 files in : 0:00:11.663727
processed 400000 files in : 0:00:11.709903
processed 500000 files in : 0:00:11.504582


In [30]:
from datetime import datetime

MAX_DOC_LENGTH = 100
DATASET_SIZE = 5000
docs = []

now = datetime.now()
file = open("/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.test.doc" , 'w')

for i, row in df.iterrows():
    file.write(f'liputan6-{str(row["id"])}' + "\n")

    sentences = list(row["sentences"][:MAX_DOC_LENGTH])
    for sentence in sentences:
        for word in sentence:
            try:
                index = str(word_to_index[word])
            except Exception as KeyError:
                index = "-1"
            file.write(index + " ")
            
        file.write("\n")

    if (i + 1) % 1000 == 0:
        print(f"processing {i + 1} with time: {datetime.now() - now}")
        now = datetime.now()

#     if (i + 1) == DATASET_SIZE:
#         break

    file.write("\n")
    
file.close()

processing 1000 with time: 0:00:00.747523
processing 2000 with time: 0:00:00.716556
processing 3000 with time: 0:00:00.734880
processing 4000 with time: 0:00:00.721491
processing 5000 with time: 0:00:00.740362
processing 6000 with time: 0:00:00.750763
processing 7000 with time: 0:00:00.744189
processing 8000 with time: 0:00:00.743272
processing 9000 with time: 0:00:00.717308
processing 10000 with time: 0:00:00.709146


In [31]:
!cat /home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.test.doc | head -c 10000

liputan6-19962
-1 164897 49805 105706 105613 2002 71427 61886 -1 128812 -1 57709 68173 162683 -1 12100 -1 -1 2002 26779 2002 49556 105613 2002 -1 79949 -1 -1 71427 105613 105706 26779 -1 162683 161519 162683 161519 71427 105706 42031 164897 -1 26779 161519 71427 2002 164897 26779 2002 71427 -1 49805 161519 49556 162683 164897 71427 105613 2002 2002 71427 -1 164897 71427 133552 105706 131123 105613 49556 164897 -1 133552 164897 -1 -1 2002 110364 2002 -1 -1 2002 49556 2002 105613 -1 42031 164897 71427 163473 163473 2002 -1 100316 164897 162683 2002 -1 105613 2002 42031 105706 71427 -1 162683 161519 71427 133552 2002 105613 2002 71427 163473 -1 12100 -1 -1 161519 49556 105613 2002 162683 164897 71427 2002 -1 2002 26779 2002 71427 -1 162683 161519 71427 133261 2002 100316 105706 49556 26779 2002 71427 -1 163473 2002 131123 -1 133261 2002 71427 163473 -1 162683 161519 71427 161519 100316 2002 71427 -1 133552 2002 71427 2002 -1 -1 -1 17020 -1 146257 151746 151746 -1 7858 105706 105613 2002 -